On-line movie recommending service using Spark & Flask 

In [1]:
!pip install pyspark

You should consider upgrading via the 'C:\Users\ASUS\anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
import pyspark
from pyspark import SparkContext
from pyspark.mllib.recommendation import ALS
import math
from flask import Flask, render_template, request
import pandas as pd
import numpy as np
from time import time

sc =SparkContext()





In [4]:
small_ratings_raw_data = sc.textFile(r"C:\Users\ASUS\Desktop\Movie recommender pyspark\dataset\ml-latest-small\ratings.csv")
small_ratings_raw_data_header = small_ratings_raw_data.take(1)[0]
small_ratings_data = small_ratings_raw_data.filter(lambda line: line!=small_ratings_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1],tokens[2])).cache()
small_ratings_data.take(3)


[('1', '1', '4.0'), ('1', '3', '4.0'), ('1', '6', '4.0')]

In [5]:

small_movies_raw_data = sc.textFile(r"C:\Users\ASUS\Desktop\Movie recommender pyspark\dataset\ml-latest-small\movies.csv")
small_movies_raw_data_header = small_movies_raw_data.take(1)[0]

small_movies_data = small_movies_raw_data.filter(lambda line: line!=small_movies_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1])).cache()
movies_titles = small_movies_data.map(lambda x: (int(x[0]),x[1]))
print ("There are %s movies in the small dataset" % (movies_titles.count()))
    


There are 9742 movies in the small dataset


In [6]:
small_movies_data.take(3)

[('1', 'Toy Story (1995)'),
 ('2', 'Jumanji (1995)'),
 ('3', 'Grumpier Old Men (1995)')]

In [7]:
training_RDD, validation_RDD, test_RDD = small_ratings_data.randomSplit([6, 2, 2], seed=0)
validation_for_predict_RDD = validation_RDD.map(lambda x: (x[0], x[1]))
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

Selecting ALS parameters using the small dataset

In [8]:
seed = 5
iterations = 10
regularization_parameter = 0.1
ranks = [4, 8, 12]
errors = [0, 0, 0]
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1
for rank in ranks:
    model = ALS.train(training_RDD, rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
    predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = validation_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    errors[err] = error
    err += 1
    print ( 'For rank %s the RMSE is %s' % (rank, error))
    if error < min_error:
        min_error = error
        best_rank = rank

print ('The best model was trained with rank %s' % best_rank)

For rank 4 the RMSE is 0.9140294340800892
For rank 8 the RMSE is 0.9239647087310089
For rank 12 the RMSE is 0.9134698936689019
The best model was trained with rank 12


In [9]:
model = ALS.train(training_RDD, best_rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
predictions = model.predictAll(test_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
rates_and_preds = test_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    
print ('For testing data the RMSE is %s' % (error))

For testing data the RMSE is 0.9160081630195371


In [10]:
complete_ratings_raw_data= sc.textFile(r"C:\Users\ASUS\Desktop\Movie recommender pyspark\dataset\ml-latest-complete\ratings.csv")
complete_ratings_raw_data_header = complete_ratings_raw_data.take(1)[0]

# Parse
complete_ratings_data = complete_ratings_raw_data.filter(lambda line: line!=complete_ratings_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),int(tokens[1]),float(tokens[2]))).cache()
    
print ("There are %s recommendations in the complete dataset" % (complete_ratings_data.count()))

There are 100836 recommendations in the complete dataset


In [11]:
complete_movies_raw_data = sc.textFile(r"C:\Users\ASUS\Desktop\Movie recommender pyspark\dataset\ml-latest-complete\movies.csv")
complete_movies_raw_data_header = complete_movies_raw_data.take(1)[0]


complete_movies_data = complete_movies_raw_data.filter(lambda line: line!=complete_movies_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),tokens[1],tokens[2])).cache()

complete_movies_titles = complete_movies_data.map(lambda x: (int(x[0]),x[1]))
    
print ("There are %s movies in the complete dataset" % (complete_movies_titles.count()))

There are 9742 movies in the complete dataset


In [12]:
training_RDD, test_RDD = complete_ratings_data.randomSplit([7, 3], seed=0)

complete_model = ALS.train(training_RDD, best_rank, seed=seed, 
                           iterations=iterations, lambda_=regularization_parameter)

In [13]:
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

predictions = complete_model.predictAll(test_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
rates_and_preds = test_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    
print ( 'For testing data the RMSE is %s' % (error))

For testing data the RMSE is 0.8970677901803009


In [14]:
training_RDD, test_RDD = complete_ratings_data.randomSplit([7, 3], seed=0)


In [15]:
def get_counts_and_averages(ID_and_ratings_tuple):
    nratings = len(ID_and_ratings_tuple[1])
    return ID_and_ratings_tuple[0], (nratings, float(sum(x for x in ID_and_ratings_tuple[1]))/nratings)

movie_ID_with_ratings_RDD = (complete_ratings_data.map(lambda x: (x[1], x[2])).groupByKey())
movie_ID_with_avg_ratings_RDD = movie_ID_with_ratings_RDD.map(get_counts_and_averages)
movie_rating_counts_RDD = movie_ID_with_avg_ratings_RDD.map(lambda x: (x[0], x[1][0]))

In [ ]:
app = Flask(__name__)


@app.route("/")
def welcome():
    return render_template('welcome_page.html')


@app.route("/rating", methods=["GET", "POST"])
def rating():
    if request.method=="POST":
        return render_template('recommendation_page.html')
    return render_template('rating.html')


@app.route("/recommendation", methods=["GET", "POST"])
def recommendation():
    if request.method == 'POST':
        
        Action = request.form.get('Action')
        Adventure = request.form.get('Adventure')
        Animation = request.form.get('Animation')
        Children = request.form.get('Children')
        Comedy = request.form.get('Comedy')
        Crime = request.form.get('Crime')
        Documentary = request.form.get('Documentary')
        Drama = request.form.get('Drama')
        Fantasy = request.form.get('Fantasy')
        Horror = request.form.get('Horror')
        Musical = request.form.get('Musical')
        Mystery = request.form.get('Mystery')
        Romance = request.form.get('Romance')
        SciFi = request.form.get('SciFi')
        Thriller = request.form.get('Thriller')
        
        new_user_ID = 0

    
        new_user_ratings = [       
                (0,1036,Action), 
                (0,3623,Adventure),
                (0,1,Animation), 
                (0,455,Children), 
                (0,6482,Comedy),
                (0,1213,Crime), 
                (0,1649,Documentary), 
                (0,858,Drama),
                (0,5816,Fantasy),
                (0,1258,Horror), 
                (0,2087,Musical),
                (0,4226,Mystery),
                (0,8533,Romance), 
                (0,260,SciFi), 
                (0,142488,Thriller) 
        ]

        new_user_ratings_RDD = sc.parallelize(new_user_ratings)
        print ('New user ratings: %s'% new_user_ratings_RDD.take(10))
        complete_data_with_new_ratings_RDD = complete_ratings_data.union(new_user_ratings_RDD)

        
        def recommendations(X, n_recommendations):
            movies['score'] = get_score(categories, preferences)
            return movies.sort_values(by=['score'], ascending=False)['title'][:n_recommendations]


        t0 = time()
        new_ratings_model = ALS.train(complete_data_with_new_ratings_RDD, best_rank, seed=seed, 
                                      iterations=iterations, lambda_=regularization_parameter)
        tt = time() - t0

        print ("New model trained in %s seconds" % round(tt,3))
        new_user_ratings_ids = map(lambda x: x[1], new_user_ratings)
        new_user_unrated_movies_RDD = (complete_movies_data.filter(lambda x: x[0] not in new_user_ratings_ids).map(lambda x: (new_user_ID, x[0])))

        new_user_recommendations_RDD = new_ratings_model.predictAll(new_user_unrated_movies_RDD)
        new_user_recommendations_rating_RDD = new_user_recommendations_RDD.map(lambda x: (x.product, x.rating))
        new_user_recommendations_rating_title_and_count_RDD = \
            new_user_recommendations_rating_RDD.join(complete_movies_titles).join(movie_rating_counts_RDD)
        new_user_recommendations_rating_title_and_count_RDD.take(3)
        new_user_recommendations_rating_title_and_count_RDD = \
        new_user_recommendations_rating_title_and_count_RDD.map(lambda r: (r[1][0][1], r[1][0][0], r[1][1]))
        top_movies = new_user_recommendations_rating_title_and_count_RDD.filter(lambda r: r[2]>=25).takeOrdered(25, key=lambda x: -x[1])

        print ('TOP recommended movies (with more than 25 reviews):\n%s' %
                '\n'.join(map(str, top_movies)))
        

        return render_template('recommendation_page.html',answers=top_movies[0:20])

if __name__ == '__main__':
   app.run(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Jan/2021 14:20:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jan/2021 14:20:35] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [12/Jan/2021 14:36:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jan/2021 14:59:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jan/2021 14:59:14] "GET /rating HTTP/1.1" 200 -


New user ratings: [(0, 1036, '3'), (0, 3623, '6'), (0, 1, '4'), (0, 455, '4'), (0, 6482, '3'), (0, 1213, '8'), (0, 1649, '4'), (0, 858, '5'), (0, 5816, '8'), (0, 1258, '5')]
New model trained in 14.35 seconds


127.0.0.1 - - [12/Jan/2021 15:01:23] "POST /recommendation HTTP/1.1" 200 -


TOP recommended movies (with more than 25 reviews):
('"Notebook', 7.2317978372479725, 38)
('Cinema Paradiso (Nuovo cinema Paradiso) (1989)', 7.173832288252788, 34)
('Harry Potter and the Chamber of Secrets (2002)', 7.010405122108389, 102)
('Harry Potter and the Order of the Phoenix (2007)', 6.912861004549992, 58)
('Serenity (2005)', 6.7432288163831995, 50)
('Harry Potter and the Prisoner of Azkaban (2004)', 6.726742570669347, 93)
('"Sound of Music', 6.666307826077401, 64)
('Harry Potter and the Deathly Hallows: Part 2 (2011)', 6.501464900530006, 50)
('"King\'s Speech', 6.501229292772595, 58)
('Big Hero 6 (2014)', 6.494582146836021, 41)
('Up (2009)', 6.480794522026132, 105)
('The Imitation Game (2014)', 6.473775951477158, 50)
("Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)", 6.4444474459693115, 107)
('Harry Potter and the Half-Blood Prince (2009)', 6.4009874863890355, 58)
('Casablanca (1942)', 6.352503882656144, 100)
('As Good as It Gets 